## Kinetic objects
The Kinetic class is responsible for calculating reaction rates and reaction enthalpies when solving the reactor equations.

In [1]:
import numpy as np

import reactord as rd

We will use an example to illustrate its functionality. First, we will define two rate constants and the volume of the reactor.

In [2]:
# Reaction 1 (constant with temperature)
kd = 0.1 / 3600 / 0.001  # m⁶/s/mol²
ki = 0.05 / 3600 / 0.001  # 1/s

# Reaction 2 (variable with temperature)
a2 = 0.15 / 3600 / 0.001  # 1/s
e2 = 1000 # K

#### Reaction rate

Next, we set the reaction rates, which are defined by python function with the format:

```python
def reaction_rate(
    composition: reactord.substance.CompositionalArgument, 
    temperature: float, 
    kinetics_constants: dict
) -> float:
    # calculation of the reaction rate 
    return evaluated_reaction_rate
```

The rate laws are defined as functions. for example, given the hypothetical reactions:
$$
2A + B \leftrightarrow C
$$

$$
C \rightarrow D
$$

Where the first one is a reversible reaction and the second is irreversible, 
and both are elemental and function of the substrates concentrations [mol/m³].
Also we have the data that, for the firs reaction, the rate of consumption of
A is:

$$r_A = k_d C_A^2 C_B - k_i C_C$$

And the rate of consumption of C en the second reaction is:

$$r_2 = a_2 e^{-e_2 / T} C_C$$

Since we have the reaction rate for the consumption of A in the first reaction.
We have to fix the stoichiometry to 1 for A:

$$
A + \frac{1}{2} B \leftrightarrow \frac{1}{2} C
$$

$$
C \rightarrow D
$$

Reaction rates are defined positively. ReactorD will automatically detect, for
a substrate, if the reaction is positive or negative. Also, in the case of 
substrate C, would take into acount the production of C by reaction 1 and the
consuption of C by reaction 2.

In [3]:
def r_rate1(concentrations, temperature, cons):
    kd, ki = cons["kd"], cons["ki"]
    
    # IMPORTANT! "A", "B", "C" are the names of our substance objects
    ca = concentrations["A"]
    cb = concentrations["B"]
    cc = concentrations["C"]
    
    return kd * ca ** 2 * cb - ki * cc


def r_rate2(concentrations, temperature, cons):
    a2, e2 = cons["a2"], cons["e2"]
    
    cc = concentrations["C"]

    return a2 * np.exp(-e2 / temperature) * cc

Note that both function follows the [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) rules of `Numpy`, this is
mandatory.

Now we must define out substances. Imagine that all the substances have the 
same molar volume and it's constant with temperature and pressure:

In [4]:
def molar_volume(temperature, pressure):
    return np.full(np.size(temperature), 1e-5) # m3/mol

Substance objects are instantiated:

NOTE THAT THE NAME OF THE SUBSTANCES IS THE SAME USED IN THE REACITON RATES!!!!

In [5]:
a = rd.Substance(name="A", volume_liquid=molar_volume)
b = rd.Substance(name="B", volume_liquid=molar_volume)
c = rd.Substance(name="C", volume_liquid=molar_volume)
d = rd.Substance(name="D", volume_liquid=molar_volume)

After that, the mixture of the substances is instantiated. In this case, we create an IdealSolution object:

In [6]:
mix = rd.mix.IdealSolution([a, b, c, d])

Next, the kinetic object is created:

In [7]:
kinetic = rd.Kinetic(
    mix=mix,
    reactions={
        "r1": {"eq": a + 1/2 * b > 1/2 * c, "rate": r_rate1},
        "r2": {"eq": c > d, "rate": r_rate2},
    },
    kinetic_constants={"kd": kd, "ki": ki, "a2": a2, "e2": e2},
    rates_argument="concentration",
)

What happend there?

Kinetic object recieves:
 * A mix object: it uses the mix object to evaluate the properties of the mixture
 In this case, molar volume, so concentrations.

 * A reactions dictionary:
    In this paramater we set the stoichiometry of our reactive system. The
    format of the dictionary is:

    ```python
    reaction_dict={"reaction_name": {"eq": algebraic_expression, "rate": rate_function}}
    ```

    The main keys of the dictionary are the reaction names and are merely 
    aesthetic.

    Then, in the interior dictionaries we have the `"eq"` key were we define 
    the stoichiometry of the reaction by algebraic operation with the 
    substance objects previously defined.

    The `"rate"` key is for the reaction rate function of that reaction.

    Finally, there is another optional key `"DH"` to specify a constant 
    reaction enthalpy for that reaction (constant with temperature and 
    pressure). If this key is not specified, ReactorD will try to calculate
    the standard reaction enthalpy from the formation enthalpies of the
    substances, and as corresponds, correct the value to temperature and
    pressure as the mixture object specifies. In this case, our reaction
    is isothermic.

    Since we have two reactions in our system, we specifies two reaction_dict.

* Kinetic constants:
    we specify the kinetic constant values with a dictionary. Note that the
    kinetic constants have the same name that it were used in the reaction
    rate functions previously defined.

* Rate argument:
    This is the compositional argument of the reaction_rate. The available
    compositional arguments in ReactorD are:  

    - Concentration $[\frac {mol} {m^3}]$  
    - Partial pressure [Pa]

    In this case, our reaction rates are function of the concentration of the
    substates, so we choose concentration.


Now we can visualy represent the kinetic object:

In [8]:
kinetic.irepr

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Also we can obtain the stoichiometric matrix of our system:

In [9]:
kinetic.stoichiometry

array([[-1. , -0.5,  0.5,  0. ],
       [ 0. ,  0. , -1. ,  1. ]])

This is why the reaction rates are defined positively. ReactorD will aply that
stoichiometry matrix to our reaction rates.

### Evaluating the kinetic

now we can evaluate both reaction rates at once, but we must specifie a mole
composition for our substances:

In [10]:
moles = np.array([10, 10, 5, 5])

mole_fractions = mix.mole_fractions(moles)

temperature = 303.15
pressure = 101325

kinetic.evaluate(mole_fractions, temperature, pressure)

array([4.34027777e+11, 3.84700593e+01])

### Constant Reaction enthalpy

To define a constant reaction enthalpy for our reaction we can set that the
first one is endothermic and has a value of 10,000 $\frac{J}{mol_A}$.
The second one is exothermic and the value of the reaciton enthalpy is
-20,000 $\frac{J}{mol_C}$.

In [11]:
kinetic = rd.Kinetic(
    mix=mix,
    reactions={
        "r1": {"eq": a + 1/2 * b > 1/2 * c, "rate": r_rate1, "DH": 10_000},
        "r2": {"eq": c > d, "rate": r_rate2, "DH": -20_000},
    },
    kinetic_constants={"kd": kd, "ki": ki, "a2": a2, "e2": e2},
    rates_argument="concentration",
)

We must initiate the reaciton enthalpy of our kinetic. This will be automatically
done for the reactors that needs this value. This features prevents that
the reactor search for values like heat capacities maybe not defined by the user
because, for example, the user want's to simulate an isothermic reactor and 
the heat capacities are not needed.

In [12]:
kinetic.set_dh_function()

kinetic.dhs_evaluate(temperature, pressure)

array([[ 10000],
       [-20000]])

With constant reaction enthalpies, doesn't matter the temperature or pressure,
the return value will remain constant.

In [13]:
temperatures = np.array([300, 400, 500, 600])

pressures = np.array([100_000, 100_000, 100_000, 100_000])

kinetic.dhs_evaluate(temperatures, pressures)

array([[ 10000,  10000,  10000,  10000],
       [-20000, -20000, -20000, -20000]])

### Variable reaction enthalpies

Now a full example with variable reaction enthalpies.

Consider the combustion of methane and ethane in gas pahse with nitrogen as
inert.

$$
CH_4 + 2 O_2 \rightarrow CO_2 + 2 H_2O
$$

$$
C_2H_6 + \frac{7}{2} O_2 \rightarrow 2 CO_2 + 3 H_2O 
$$

For the example are used toy reaction rates.

For the calculation of the reaction enthalpies are necessary the standard 
ideal gas formation enthalpies of all the substrates, and the ideal gas 
heat capacities. For our luck, we can take them from the thermo library.


In [18]:
# =============================================================================
# Substances
# =============================================================================
ch4 = rd.Substance.from_thermo_database("ch4", "methane")
o2 = rd.Substance.from_thermo_database("o2", "oxygen")
co2 = rd.Substance.from_thermo_database("co2", "carbon dioxide")
h2o = rd.Substance.from_thermo_database("h2o", "water")
c2h6 = rd.Substance.from_thermo_database("c2h6", "ethane")
# DON'T FORGET THE INERT ;)
n2 = rd.Substance.from_thermo_database("n2", "nitrogen")

# =============================================================================
# Mixture: now a ideal gas
# =============================================================================
mix = rd.mix.IdealGas([ch4, o2, co2, h2o, c2h6, n2])

# =============================================================================
# Reaction rates
# =============================================================================
def r1(c, t, cons):
    k1 = cons["k1"]
    c_ch4, c_o2 = c["ch4"], c["o2"]
    
    return k1 * c_ch4 * c_o2

def r2(c, t, cons):
    k2 = cons["k2"]
    c_c2h6, c_o2 = c["c2h6"], c["o2"]
    
    return k2 * c_c2h6 * c_o2

# =============================================================================
# Kinetic object
# =============================================================================
kinetic = rd.Kinetic(
    mix=mix,
    reactions={
        "r1": {"eq": ch4 + 2 * o2 > co2 + 2 * h2o, "rate": r1},
        "r2": {"eq": c2h6 + 7/2 * o2 > 2 * co2 + 3 * h2o, "rate": r2}
    },
    kinetic_constants={"k1": 1e-5, "k2": 1e-6},
    rates_argument="concentration" 
)

# Start the reaction enthalpy function
kinetic.set_dh_function()

In [16]:
kinetic.stoichiometry

array([[-1. , -2. ,  1. ,  2. ,  0. ,  0. ],
       [ 0. , -3.5,  2. ,  3. , -1. ,  0. ]])

In [21]:
moles = np.array(
    [
        [10, 15, 16], # ch4 moles
        [20, 10, 16], # o2 moles
        [50, 20, 30], # co2 moles
        [0, 20, 15], # h2o moles
        [40, 20, 30], # c2h6 moles
        [10, 0, 30] # n2 moles
    ]
)

mole_fractions = mix.mole_fractions(moles)
temperatures = np.array([500, 1000, 2000])
pressures = np.array([100_000, 200_000, 300_000])


kinetic.evaluate(mole_fractions, temperatures, pressures)

array([[6.84754527e-05, 1.20128216e-04, 4.43928451e-05],
       [2.73901811e-05, 1.60170955e-05, 8.32365845e-06]])

In [22]:
kinetic.dhs_evaluate(temperatures, pressures)

array([[ -800832.99199485,  -801075.31914692,  -809803.25400112],
       [-1425727.62523699, -1427790.25945566, -1443262.71596654]])